## loading file

In [2]:
import os
print(os.getcwd())


c:\Users\aalqarawi.t\OneDrive - Solutions by stc\Desktop\chatbot\notebook


In [8]:
import os
import pandas as pd

# Get the base folder of your project (two levels up from /notebook)
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__))) if "__file__" in globals() else os.path.dirname(os.getcwd())

# Build the path to the Excel file inside /data
DATA_PATH = os.path.join(BASE_DIR, "data", "amazon_product_reviews.xlsx")

# Load the file
df = pd.read_excel(DATA_PATH)
print("Data loaded successfully from the 'data' folder!")
print(f"Initial shape of the DataFrame: {df.shape}")
print("\nFirst 5 rows of the dataset:")
print(df.head())



Data loaded successfully from the 'data' folder!
Initial shape of the DataFrame: (1465, 16)

First 5 rows of the dataset:
   product_id                                       product_name  \
0  B07JW9H4J1  Wayona Nylon Braided USB to Lightning Fast Cha...   
1  B098NS6PVG  Ambrane Unbreakable 60W / 3A Fast Charging 1.5...   
2  B096MSW6CT  Sounce Fast Phone Charging Cable & Data Sync U...   
3  B08HDJ86NZ  boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...   
4  B08CF3B7N1  Portronics Konnect L 1.2M Fast Charging 3A 8 P...   

                                            category discounted_price  \
0  Computers&Accessories|Accessories&Peripherals|...           â‚¹399   
1  Computers&Accessories|Accessories&Peripherals|...           â‚¹199   
2  Computers&Accessories|Accessories&Peripherals|...           â‚¹199   
3  Computers&Accessories|Accessories&Peripherals|...           â‚¹329   
4  Computers&Accessories|Accessories&Peripherals|...           â‚¹154   

  actual_price  discount_perce